In [29]:
# for colab environment
from google.colab import drive
!ls /content/drive/MyDrive/projects/ML_FinalProj
import sys
import os

drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/projects/ML_FinalProj')

os.chdir("/content/drive/MyDrive/projects/ML_FinalProj")

DataPreprocessing.ipynb  MLProject_Progress.ipynb     zips
dataset			 MLTeamPredictionBasic.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path("dataset")
# import
TEAM_STAT_PATH = ROOT / "team-stat"
GAME_LOG_PATH = ROOT / "game-log"
PITCHER_PATH = ROOT / "pitcher" / "sp_yearly_game"
TOTAL_TEAM_STAT_PATH = TEAM_STAT_PATH / "01-25_팀기록.csv"
# export
PROCESSED_PATH = ROOT / "processed"
final_dataset_path = PROCESSED_PATH / "final_dataset_v1.csv"
team_features_path = PROCESSED_PATH / "team_features_v1.csv"
final_base_path = PROCESSED_PATH / "final_dataset_no_diff_v1.csv"
final_dataset_path = PROCESSED_PATH / "final_dataset_with_diff_v1.csv"

PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

In [31]:
# 팀명->id 전처리용
# 코드 전체에서 동일한 mapping 적용
team_groups = {
    0: ['KIA', '해태', '해태타이거즈', '해태 타이거즈', '기아', '기아타이거즈', '기아 타이거즈', 'KIA 타이거즈', 'KIA타이거즈'],
    1: ['삼성', '삼성라이온즈', '삼성 라이온즈'],
    2: ['두산', '두산베어스', '두산 베어스'],
    3: ['LG', 'LG 트윈스', 'LG트윈스'],
    4: ['KT', 'KT 위즈', 'KT위즈'],
    5: ['SSG', 'SK', 'SSG랜더스', 'SSG 랜더스', 'SK와이번스', 'SK 와이번스'],
    6: ['롯데', '롯데자이언츠', '롯데 자이언츠'],
    7: ['한화', '한화이글스', '한화 이글스'],
    8: ['NC', 'NC다이노스', 'NC 다이노스'],
    9: ['키움', '넥센', '우리', '히어로즈', '키움 히어로즈', '키움히어로즈', '우리 히어로즈', '우리히어로즈', '넥센 히어로즈', '넥센히어로즈'],
    10: ['현대', '현대유니콘스', '현대 유니콘스']
}

In [32]:
# 팀기록 처리

team_stat_df = pd.read_csv(TOTAL_TEAM_STAT_PATH)
teams = team_stat_df["team"]
team_names = [t[2:] for t in teams]
unique_team_names = np.unique(team_names)


team_stat_df_edited = team_stat_df.drop(columns=["wRC+.y", "G.x", "G.y", "G.x.x", "G.y.y", "HR.y", "R/ePA.y",
                                                 "IsoP.y", "ERA.y", "RA9.y", "FIP.y", "rRA9.y"]) # 중복제거
team_stat_df_edited = team_stat_df_edited.drop(columns=["POSAdj", "RAAwithPOS", "WAAwithPOS", "HR/FB", "WAR당 연봉.y"]) # 결측치 존재하는 column 제거
# 일부 column 이름 정리
team_stat_df_edited = team_stat_df_edited.rename(columns={"wRC+.x": "wRC+"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"R/ePA.x": "R/ePA"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"IsoP.x": "IsoP"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"ERA.x": "ERA"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"RA9.x": "RA9"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"FIP.x": "FIP"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"rRA9.x": "rRA9"})
team_stat_df_edited = team_stat_df_edited.rename(columns={"WAR당 연봉.x": "WAR당 연봉"})

# 팀 이름 / 연도 분리
team_stat_df_edited.insert(loc=1, column="year", value=team_stat_df_edited["team"].str[:2])
team_stat_df_edited.insert(loc=2, column="name", value=team_stat_df_edited["team"].str[2:])

# 팀 이름 id 할당
name_to_id = {name: group_id for group_id, names in team_groups.items() for name in names} # 팀 이름 -> id 변환 mapping
team_stat_df_edited.insert(loc=0, column='id', value=team_stat_df_edited['name'].map(name_to_id))
team_stat_df_edited = team_stat_df_edited.drop(columns=["team"])


In [33]:
# id, year별로 year-1년까지의 모든 지표 가중합하여 전처리
def build_decayed_team_features(df, decay, exclude_cols = ("id", "year", "name", "G")):
  suffix = "_decay"

  df_out = df.copy()
  df_out["year"] = pd.to_numeric(df_out["year"], errors='raise')
  df_out = df_out.sort_values(["id", "year"], kind="mergesort")
  cand_cols = [c for c in df_out.columns if c not in exclude_cols]

  # 연도 고려하여 각 지표의 decay 누적 계산
  # input: pd.DataFrame     output: pd.Series
  def _decayed_cumsum_with_gaps(sub, col):
    years = sub["year"].to_numpy()
    vals = pd.to_numeric(sub[col], errors="coerce").fillna(0).to_numpy(dtype=float)
    out = np.zeros_like(vals)
    s = 0.0
    prev_year = None
    for i in range(len(vals)):
      if prev_year is None:
        gap_decay = 1.0
      else:
        gap = int(years[i] - prev_year)
        gap_decay = decay ** max(1, gap)
      prev_val = vals[i-1] if i>0 else 0.0
      s = s * gap_decay + prev_val
      out[i] = s
      prev_year = years[i]
    return pd.Series(out, index=sub.index)

  new_cols_list = []

  for col in cand_cols:
    new_col = f"{col}{suffix}"

    new_series = (
        df_out.groupby("id", group_keys=False).apply(
            lambda x: _decayed_cumsum_with_gaps(x, col),
            include_groups=False
        )
    )
    new_series.name = new_col
    new_cols_list.append(new_series)
  df_out = pd.concat([df_out] + new_cols_list, axis=1)
  return df_out

In [34]:
# game-log csv 불러오기 및 정렬
game_log_csv_names = ["games_20" + str(i).zfill(2) + ".csv" for i in range(2, 26)]
game_log_df = pd.DataFrame()

for i in range(2, 26):
  GAME_LOG_PATH_PER_YEAR = GAME_LOG_PATH / game_log_csv_names[i-2]
  game_log_per_year_df = pd.read_csv(GAME_LOG_PATH_PER_YEAR)
  game_log_df = pd.concat([game_log_df, game_log_per_year_df])

# game-log에서 팀 이름을 id로 변환
game_log_df = game_log_df.assign(
    home_id = game_log_df["home_team"].map(name_to_id),
    away_id = game_log_df["away_team"].map(name_to_id),
)

### 해당 시즌 내 지표(~승률, 최근10경기) 계산
game_log_base = game_log_df[['date', 'season', 'home_id', 'away_id', 'home_runs', 'away_runs']].dropna(subset=['home_id', 'away_id'])
game_log_base['home_id'] = game_log_base['home_id'].astype(int)
game_log_base['away_id'] = game_log_base['away_id'].astype(int)

home_games = game_log_base.rename(columns={'home_id': 'team_id', 'away_id': 'opp_id'})
home_games['win'] = (home_games['home_runs'] > home_games['away_runs']).astype(int)
away_games = game_log_base.rename(columns={'away_id': 'team_id', 'home_id': 'opp_id'})
away_games['win'] = (away_games['away_runs'] > away_games['home_runs']).astype(int)

all_games_df = pd.concat([
    home_games[['date', 'season', 'team_id', 'win']],
    away_games[['date', 'season', 'team_id', 'win']]
])
all_games_df = all_games_df.sort_values(by=['team_id', 'date'])

# 해당 시즌 누적 승률(직전 경기까지 반영)
gb_season = all_games_df.groupby(['team_id', 'season'])
wins_season_total = gb_season['win'].cumsum()
games_season_total = gb_season.cumcount() + 1
prev_wins = wins_season_total.shift(1).fillna(0)
prev_games = games_season_total.shift(1).fillna(0)
all_games_df['in_season_win_rate'] = (prev_wins / prev_games).fillna(0)

# 최근 10경기 승률
gb_team = all_games_df.groupby('team_id')
all_games_df['L10_win_rate'] = gb_team['win'].shift(1).rolling(
    window=10, min_periods=1
).mean().reset_index(level=0, drop=True).fillna(0)

# 병합
stats_to_merge = all_games_df.drop_duplicates(
    subset=['date', 'season', 'team_id'], keep='last'
)
stats_to_merge = stats_to_merge[['date', 'season', 'team_id', 'in_season_win_rate', 'L10_win_rate']]

# 지표별 decay 적용
team_stat_df_edited = build_decayed_team_features(
    team_stat_df_edited,
    decay=0.9
)
feature_cols = ['id', 'year'] + [col for col in team_stat_df_edited.columns if col.endswith("_decay")]
team_features_df = team_stat_df_edited[feature_cols].copy()

home_features = team_features_df.rename(columns={
    col: f"home_{col}" for col in team_features_df.columns if col not in ['id', 'year']
})
away_features = team_features_df.rename(columns={
    col: f"away_{col}" for col in team_features_df.columns if col not in ['id', 'year']
})

# game-log 전처리
game_log_df_cleaned = game_log_df.dropna(subset=["home_id", "away_id"]).copy()
game_log_df_cleaned['year'] = (game_log_df_cleaned['season'] % 100).astype(int)
game_log_df_cleaned['home_id'] = game_log_df_cleaned['home_id'].astype(int)
game_log_df_cleaned['away_id'] = game_log_df_cleaned['away_id'].astype(int)
game_log_df_cleaned['home_win'] = (game_log_df_cleaned['home_runs'] > game_log_df_cleaned['away_runs']).astype(int)
game_data_df = game_log_df_cleaned[['date', 'season', 'year', 'home_id', 'away_id', 'home_runs', 'away_runs', 'home_win']]

# game-log와 연도별 지표 결합
merged_df = pd.merge(
    game_data_df, home_features,
    left_on=['home_id', 'year'], right_on=['id', 'year'], how='left'
)
final_df = pd.merge(
    merged_df, away_features,
    left_on=['away_id', 'year'], right_on=['id', 'year'], how='left'
)
final_df = final_df.drop(columns=['id_x', 'id_y'])

# 해당 연도 승률 지표 추가
final_df_with_momentum = pd.merge(
    final_df,
    stats_to_merge.rename(columns={
        'team_id': 'home_id',
        'in_season_win_rate': 'home_in_season_win_rate',
        'L10_win_rate': 'home_L10_win_rate'
    }),
    on=['date', 'season', 'home_id'], how='left'
)
final_df_with_momentum = pd.merge(
    final_df_with_momentum,
    stats_to_merge.rename(columns={
        'team_id': 'away_id',
        'in_season_win_rate': 'away_in_season_win_rate',
        'L10_win_rate': 'away_L10_win_rate'
    }),
    on=['date', 'season', 'away_id'], how='left'
)

# 결측치 처리
new_momentum_cols = ['home_in_season_win_rate', 'home_L10_win_rate', 'away_in_season_win_rate', 'away_L10_win_rate']
final_df_with_momentum[new_momentum_cols] = final_df_with_momentum[new_momentum_cols].fillna(0)

# 두 팀의 차이 지표
final_df_with_diff = final_df_with_momentum.copy()

# diff에 decay 적용
decay_cols = [col for col in team_features_df.columns if col.endswith('_decay')]
base_cols = [c for c in decay_cols if c not in ['id', 'year']]
home_mat = final_df_with_diff[[f'home_{c}' for c in base_cols]].copy()
home_mat.columns = base_cols
away_mat = final_df_with_diff[[f'away_{c}' for c in base_cols]].copy()
away_mat.columns = base_cols
diff_arr = home_mat.values - away_mat.values
diff_cols = [f'diff_{c}' for c in base_cols]
diff_df = pd.DataFrame(diff_arr, columns=diff_cols, index=final_df_with_diff.index)
final_df_with_diff = pd.concat([final_df_with_diff, diff_df], axis=1, copy=False)

final_df_with_diff['diff_in_season_win_rate'] = final_df_with_diff['home_in_season_win_rate'] - final_df_with_diff['away_in_season_win_rate']
final_df_with_diff['diff_L10_win_rate'] = final_df_with_diff['home_L10_win_rate'] - final_df_with_diff['away_L10_win_rate']

final_df_with_diff = final_df_with_diff.copy()
final_df_filtered = final_df_with_diff[final_df_with_diff['season'] > 2002].copy()
final_df_complete = final_df_filtered.dropna() # 최종 df


In [35]:
# starter csv 불러오기 및 병합
starters_csv_names = [f"kbo_starters_regular_season_{year}.csv" for year in range(2002, 2026)]
starters_df = pd.DataFrame()

for year in range(2002, 2026):
    CURRENT_PITCHER_PATH = PITCHER_PATH / f"kbo_starters_regular_season_{year}.csv"

    if CURRENT_PITCHER_PATH.exists():
        temp_df = pd.read_csv(CURRENT_PITCHER_PATH)
        starters_df = pd.concat([starters_df, temp_df], ignore_index=True)
    else:
        print(f"File not found: {CURRENT_PITCHER_PATH}")

# 팀 이름 전처리 적용
starters_df['team_id'] = starters_df['team'].map(name_to_id)

In [36]:
# column 추출 및 처리
starters_subset = starters_df[['date', 'team_id', 'era']].copy()
starters_subset['era'] = pd.to_numeric(starters_subset['era'], errors='coerce')
final_df_complete['date'] = final_df_complete['date'].astype(str)
starters_subset['date'] = starters_subset['date'].astype(str)
starters_subset = starters_subset.drop_duplicates(subset=['date', 'team_id'], keep='first')

# home era 병합
final_df_complete = pd.merge(
    final_df_complete,
    starters_subset.rename(columns={'team_id': 'home_id', 'era': 'home_era'}),
    on=['date', 'home_id'],
    how='left'
)

# away era 병합
final_df_complete = pd.merge(
    final_df_complete,
    starters_subset.rename(columns={'team_id': 'away_id', 'era': 'away_era'}),
    on=['date', 'away_id'],
    how='left'
)

# diff 계산
final_df_complete['diff_era'] = final_df_complete['home_era'] - final_df_complete['away_era']

# 결측치 처리
print(f"Before dropping missing pitcher data: {final_df_complete.shape}")
final_df_complete = final_df_complete.dropna(subset=['home_era', 'away_era'])
print(f"After dropping missing pitcher data: {final_df_complete.shape}")


Before dropping missing pitcher data: (14279, 392)
After dropping missing pitcher data: (14272, 392)


In [37]:
# 최종 dataset
final_df_complete.to_csv(final_dataset_path, index=False, encoding='utf-8-sig')

# diff 없는 dataset
final_base_df = final_df_with_momentum[final_df_with_momentum['season'] > 2002].dropna()
final_base_df.to_csv(final_base_path, index=False, encoding='utf-8-sig')